# ===============================================================
#  AI-BASED MEETING ANALYZER
# ===============================================================

# 📌 Project Title:
    AI-Based Meeting Analyzer

# 📋 Description:
    This application automatically transcribes any meeting audio
    (MP3, WAV, etc.) into text, then uses LLMs to:
      • Generate concise and structured summaries
      • Extract decisions, responsible parties, and deadlines
      • Classify meeting type and extract dates
      • Provide an interactive Q&A chat over the transcript

# 🤝 Collaboration:
    Built in partnership with the Saudi Scientific Home
    Healthcare Society (SSHHS)

# 🛠️ Technologies & Frameworks:
    • Speech-to-Text: OpenAI Whisper (via faster-whisper)
    • Summarization: HuggingFace distilBART (pipeline)
    • LLM & RAG: LangChain + OpenAI’s Chat models + Chroma
    • UI: Gradio
    • Evaluation: LangSmith tracing

# ⚙️ Requirements:
    • Python ≥ 3.8
    • torch, pydub, transformers, langchain, chromadb, gradio
    • Valid OPENAI_API_KEY & LANGCHAIN_API_KEY in .env

# 🚀 Usage:
    1. Set your API keys in a `.env` file.
    2. Run `python app.py` and open the Gradio link.
    3. Upload your audio file and ask questions in the chat.

# ===============================================================


In [1]:
# Install required packages for the Smart Meeting Assistant:
# - gradio: UI framework
# - faster-whisper, pydub: audio transcription
# - python-dotenv: environment variable loading
# - sentence-transformers, transformers: NLP models
# - sympy: math utilities (LangChain dependency)
# - chromadb: vector database
# - langchain-* packages: core LangChain functionality and community extensions
!pip install -q --no-cache-dir \
  gradio \
  faster-whisper \
  pydub \
  python-dotenv \
  sentence-transformers \
  "transformers==4.43.0" \
  sympy==1.13.1 \
  chromadb \
  langchain-openai \
  langchain-community \
  langchain-huggingface


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 34.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 267.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 362.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 369.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 258.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 324.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 312.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 344.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 270.5 MB/s 

In [2]:
# ===============================================================
#  Smart Meeting Assistant – universal audio-to-text workflow
# ===============================================================

import os, re, shutil, traceback, gradio as gr              # OS ops, regex, file handling, error traces, Gradio UI
import torch                                                # Check for GPU/CPU
from dotenv import load_dotenv                              # Load .env variables
from pydub import AudioSegment                              # Slice and export audio files
from faster_whisper import WhisperModel                     # Whisper speech-to-text wrapper
from transformers import pipeline, AutoTokenizer            # HF pipelines & tokenizers
from langchain_openai import ChatOpenAI                     # LangChain OpenAI wrapper
from langchain_huggingface import HuggingFaceEmbeddings     # Sentence-transformers embeddings
from langchain.document_loaders import TextLoader           # Read text files as LangChain docs
from langchain_community.vectorstores import Chroma         # Local Chroma vector DB
from langchain.chains import RetrievalQA                    # Retriever-augmented QA chain
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Split long text safely

# ===============================================================
#  Step 1: Load environment & LangSmith tracing setup
# ===============================================================
load_dotenv()                                              # 1️⃣ Load .env first
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Please set OPENAI_API_KEY in .env"

# 2️⃣ Configure LangSmith tracing via both ENV and direct client API key
os.environ.setdefault("LANGCHAIN_TRACING_V2", "true")      # enable v2 tracing
from langsmith.client import Client                        # explicit import path
from langchain.callbacks.tracers.langchain import LangChainTracer

LS_API_KEY = os.getenv("LANGSMITH_API_KEY", "YOUR_LANGSMITH_KEY")
                                                          # prefer .env, fallback to literal
client = Client(api_key=LS_API_KEY)                       # pass key explicitly
tracer = LangChainTracer(project_name="smart-meeting-assistant")
print("✅ LangSmith tracing is enabled!")
print("✅ View your runs at: https://smith.langchain.com")

# ===============================================================
#  Step 2: Define working directories
# ===============================================================
WORK_DIR     = "/content"
CHROMA_DIR   = os.path.join(WORK_DIR, "chroma_db")
MEETING_FILE = os.path.join(WORK_DIR, "meeting.txt")

# ===============================================================
#  Step 3: Initialize models and tokenizers
# ===============================================================
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"
COMPUTE_TYPE = "float16" if DEVICE == "cuda" else "int8"
whisper      = WhisperModel("tiny", device=DEVICE, compute_type=COMPUTE_TYPE)

SUM_MODEL    = "sshleifer/distilbart-cnn-6-6"
tok          = AutoTokenizer.from_pretrained(SUM_MODEL)
summarizer   = pipeline("summarization", model=SUM_MODEL, device=-1)

chat_llm     = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
embedder     = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ===============================================================
#  Step 4: Split & transcribe
# ===============================================================
def split_audio(path: str, ms: int = 120_000) -> list[str]:
    """Slice audio into <=2-min chunks."""
    audio = AudioSegment.from_file(path)
    names = []
    fmt   = os.path.splitext(path)[1].lstrip(".") or "mp3"
    for i in range(0, len(audio), ms):
        fn = os.path.join(WORK_DIR, f"chunk_{i//ms}.{fmt}")
        audio[i:i+ms].export(fn, format=fmt); names.append(fn)
    return names

def transcribe(parts: list[str]) -> str:
    """Run Whisper on each chunk, concat text, delete files."""
    txt = ""
    for fp in parts:
        segs, _ = whisper.transcribe(fp, beam_size=5)
        if segs: txt += " " + " ".join(s.text for s in segs)
        os.remove(fp)
    return txt.strip()

# ===============================================================
#  Step 5: Summarize long text
# ===============================================================
def summarize_long(txt: str, chunk_size: int = 1000, chunk_overlap: int = 200) -> str:
    """Split into overlapping chunks, summarize each, then merge."""
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
    )
    chunks = splitter.split_text(txt)
    out    = []
    for c in chunks:
        try:
            summary = summarizer(c, max_length=160, min_length=40, do_sample=False)[0]["summary_text"]
        except:
            summary = c[:500] + ("…" if len(c)>500 else "")
        out.append(summary)
    if len(out) > 1:
        mega = " ".join(out)
        try:
            return summarizer(mega, max_length=160, min_length=40, do_sample=False)[0]["summary_text"]
        except:
            return mega[:1000] + ("…" if len(mega)>1000 else "")
    return out[0]

# ===============================================================
#  Step 6: Classify meeting type
# ===============================================================
def classify_meeting(txt: str) -> str:
    """Prompt LLM to label meeting type in ≤30 chars."""
    prompt = f"Classify this meeting in ≤30 chars (e.g., Finance Mtg):\n\n{txt[:1000]}"
    return chat_llm.invoke(prompt).content.strip()

# ===============================================================
#  Step 7: Extract date
# ===============================================================
def extract_date(txt: str) -> str:
    """Grab ISO dates or fallback to 'Mon DD' regex."""
    iso = re.findall(r"\b\d{4}-\d{2}-\d{2}\b", txt)
    if iso: return iso[0]
    alt = re.findall(r"\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\w* \d{1,2}\b", txt)
    return alt[0] if alt else "No date mentioned"

# ===============================================================
#  Step 8: Structured summary
# ===============================================================
def structured_summary(txt: str) -> str:
    """
    Extract:
    1. Decisions
    2. Responsible persons + tasks
    3. Deadlines
    4. Additional notes
    Enforce Markdown bullets, no hallucinations.
    """
    prompt = (
        "You are a specialized assistant for extracting structured meeting minutes.\n"
        "Extract:\n1. Decisions\n2. Responsible persons + tasks\n"
        "3. Deadlines\n4. Additional notes\n"
        "- Use Markdown bullets (- ). No invented info.\n"
        "Transcript:\n" + txt[:4000]
    )
    return chat_llm.invoke(prompt).content.strip()

# ===============================================================
#  Step 9: Build RAG DB
# ===============================================================
def build_db(txt: str) -> Chroma:
    """Rebuild vector DB from the full transcript."""
    if os.path.exists(CHROMA_DIR): shutil.rmtree(CHROMA_DIR)
    os.makedirs(CHROMA_DIR, exist_ok=True)
    with open(MEETING_FILE, "w", encoding="utf-8") as f:
        f.write(txt)
    docs = TextLoader(MEETING_FILE).load()
    return Chroma.from_documents(docs, embedder, persist_directory=CHROMA_DIR)

# ===============================================================
#  Step 10: Gradio – process upload
# ===============================================================
def process_audio(file_obj, state):
    """
    ● Transcribe uploaded audio
    ● Generate short + structured summaries, type, date
    ● Instantiate a LangSmith‐traced RAG agent
    """
    try:
        transcript = transcribe(split_audio(file_obj.name))
        short_sum  = summarize_long(transcript)
        struct_sum = structured_summary(transcript)
        mtype      = classify_meeting(transcript)
        mdate      = extract_date(transcript)
        agent = RetrievalQA.from_chain_type(
            llm=chat_llm,
            retriever=build_db(transcript).as_retriever(search_kwargs={"k":3}),
            callbacks=[tracer]
        )
        state["agent"] = agent
        return transcript, short_sum, struct_sum, mtype, mdate, state

    except Exception as e:
        traceback.print_exc()
        err = f"❌ ERROR: {e}"
        return err, err, err, err, err, state

# ===============================================================
#  Step 11: Gradio – chat interface
# ===============================================================
def chat_fn(msg, history, state):
    """
    ● Send user query to the stored RAG agent
    ● Append user & assistant messages to history
    """
    agent = state.get("agent")
    if not agent:
        history.append({"role":"assistant","content":"⚠️ Please process audio first."})
        return history, state

    resp   = agent.invoke({"query": msg})
    answer = resp["result"].strip()
    history += [
        {"role":"user",     "content":msg},
        {"role":"assistant","content":answer}
    ]
    return history, state

# ===============================================================
#  Step 12: Launch Gradio UI
# ===============================================================
demo = gr.Blocks(title="Smart Meeting Assistant")
with demo:
    gr.Markdown("### 🎙️ Smart Meeting Minutes Assistant – universal audio-to-text")
    st    = gr.State({})
    file  = gr.File(label="🎧 Upload audio (MP3, WAV...)")
    btn   = gr.Button("🚀 Transcribe & Summarize")
    tr    = gr.Textbox(label="📝 Transcript",          lines=8)
    summ  = gr.Textbox(label="🧠 Short Summary",       lines=4)
    struct= gr.Textbox(label="📋 Structured Summary",  lines=6)
    typ   = gr.Textbox(label="📌 Meeting Type")
    date  = gr.Textbox(label="📅 Meeting Date")
    chat  = gr.Chatbot(label="🤖 Ask About the Meeting", type="messages")
    ask   = gr.Textbox(placeholder="Ask a question about the meeting…")

    btn.click(process_audio, inputs=[file, st], outputs=[tr, summ, struct, typ, date, st])
    ask.submit(chat_fn,       inputs=[ask, chat, st], outputs=[chat, st]).then(lambda: "", None, ask)

demo.launch()  # Start the Gradio server


✅ LangSmith tracing is enabled!
✅ View your runs at: https://smith.langchain.com


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5660e723807af57c8e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
